In [ ]:
%pip install folium.plugins --upgrade
%pip install folium
%pip install shapely
%pip install branca
%pip install matplotlib

In [ ]:
import json
import folium
import pandas as pd
from shapely.geometry import LineString, MultiLineString, Point, mapping
from shapely.wkt import loads
from matplotlib.colors import rgb2hex           
from branca.colormap import linear

##### Coordinates for each center

In [ ]:
dictionary_coord = {}
dictionary_coord['FIB'] = (41.389509464271306, 2.1132900949612536)
dictionary_coord['ETSEIB'] = (41.384880843955735, 2.1155950630696103)
dictionary_coord['ETSAB'] = (41.38405501716756, 2.1140366968134603)
dictionary_coord['ETSETB'] = (41.388896550663965, 2.1122289968137182)
dictionary_coord['ETSECCPB'] = (41.388038743808714, 2.1110871956118213)
dictionary_coord['ESEIAAT'] = (41.56311953197128, 2.0230346054568247)
dictionary_coord['ETSAV'] = (41.47032499138291, 2.0705851263674204)
dictionary_coord['EPSEM'] = (41.7367619, 1.8275600093754318)
dictionary_coord['EPSEB'] = (41.38398159574671, 2.112779422594055)
dictionary_coord['EEBE'] = (41.41358164329492, 2.2220088802759044)
dictionary_coord['EEABB'] = (41.2765983603705, 1.9861430774420645)
dictionary_coord['EETAC'] = (41.27568749859876, 1.9872608121524569)
dictionary_coord['EPSEVG'] = (41.22150491967886, 1.729683450772624)
dictionary_coord['FOOT'] = (41.56874095856424, 2.0236936208029763)
dictionary_coord['FNB'] = (41.38319996329999, 2.183814121174014)
dictionary_coord['FME'] = (41.38438382368992, 2.1157410269356474)

##### Styles for each type of feature

In [ ]:
def style_bicis(feature):
    return {
        'fillColor': 'violet',
        'color': 'violet',
        'weight': 2,
        'fillOpacity': 0.5,
    }

In [ ]:
df = pd.read_csv("coordenades2.csv", delimiter=",")
map_center = [41.38879, 2.15899]
map_multiPoints = folium.Map(location=map_center, zoom_start=14)
# Function to add geometries to the map
def add_to_map(geometry, color):
    if geometry.geom_type == 'MultiLineString':
        for line in geometry.geoms:  # Iterate over individual LineString components
            if (color=="blue") :
                folium.PolyLine(locations=[(lat, lon) for lon, lat in line.coords], color=color, opacity=1).add_to(map_multiPoints)
            else:
                folium.PolyLine(locations=[(lat, lon) for lon, lat in line.coords], color=color, opacity=1).add_to(map_multiPoints)
    elif geometry.geom_type == 'Point':
        lat, lon = geometry.y, geometry.x
        folium.CircleMarker(location=[lat, lon], radius=3, color=color, fill=True).add_to(map_multiPoints)

In [ ]:
# Iterate through rows and add geometries to the map
for index, row in df.iterrows():
    for column in df.columns:
        if 'geometry' in column and pd.notna(row[column]) and row[column] != "":
            geometry = loads(str(row[column]))
            if('metro' in column) :
                add_to_map(geometry, "brown")
            else :
                pass#add_to_map(geometry, "brown")

with open('biciRoutes.geojson') as f:
    geojson_data = json.load(f)

# Iterate through features in the GeoJSON file
# Add GeoJSON data to the map
folium.GeoJson(geojson_data, style_function=style_bicis).add_to(map_multiPoints)



##### Plotting bike spots

In [ ]:
df_bicis = pd.read_csv("bicis_coordinades.csv", delimiter=",")
for row in df_bicis.iterrows() :
    folium.CircleMarker(location=[row[1]['latitude_bicing'], row[1]['longitude_bicing']], radius=3, color="violet", fill=True, fill_opacity=0.8).add_to(map_multiPoints)

##### For every postal code, plot the data of the students from there

In [ ]:
pre_df = pd.read_csv("preprocessed.csv", delimiter=",")

mapa = {}

for _, item in pre_df[["postal", "Latitude", "Longitude", "private_ratio", "public_ratio","active_ratio"]].iterrows():
    if str(item[0]) not in mapa:
        mapa[str(item[0])] = [0, 0, 0, 0, 0, 0]
        mapa[str(item[0])][4] = item[1]
        mapa[str(item[0])][5] = item[2]
    mapa[str(item[0])][0] += 1
    mapa[str(item[0])][1] += item[3]
    mapa[str(item[0])][2] += item[4]
    mapa[str(item[0])][3] += item[5]
print(mapa)
for value in mapa.values() :
    if(value[1] == max(value[1], value[2], value[3])) : 
        folium.CircleMarker(location=[value[4], value[5]], radius=value[0] * 1.01, color="blue", fill=True, fill_opacity=0.8).add_to(map_multiPoints)
    elif(value[2] == max(value[1], value[2], value[3])) : 
        folium.CircleMarker(location=[value[4], value[5]], radius=value[0] * 1.01, color="orange", fill=True, fill_opacity=0.8).add_to(map_multiPoints)
    else : 
        folium.CircleMarker(location=[value[4], value[5]], radius=value[0] * 1.01, color="green", fill=True, fill_opacity=0.8).add_to(map_multiPoints)

##### Plot the HeatMap

In [ ]:
from folium.plugins import HeatMap

pre_df = pd.read_csv("preprocessed.csv", delimiter=",")

mapa = {}

for _, item in pre_df[["postal", "Latitude", "Longitude", "private_ratio", "public_ratio","active_ratio"]].iterrows():
    if str(item[0]) not in mapa:
        mapa[str(item[0])] = [0, 0, 0, 0, 0, 0]
        mapa[str(item[0])][4] = item[1]
        mapa[str(item[0])][5] = item[2]
    mapa[str(item[0])][0] += 1
    mapa[str(item[0])][1] += item[3]
    mapa[str(item[0])][2] += item[4]
    mapa[str(item[0])][3] += item[5]

HeatMap([[value[1][4], value[1][5], value[1][0]] for value in mapa.items()]).add_to(map_multiPoints)

##### Declaration of the function to draw lines

In [ ]:
import folium.plugins

def draw_arrow(map, start, end, arrow_scale=2, arrow_color = 200):
    # Line with size arrow_scale
    # Make a blue with the value of arrow_color
    color = f"#{0}{0}{arrow_color:02x}"
    folium.PolyLine([start, end], color=color, weight=arrow_scale, border_color="black", border_width=2,).add_to(map)
    arrow = folium.Marker(location=end,
                          icon=folium.plugins.BeautifyIcon(
                              icon='arrow-up',
                              border_color="yellow",
                              border_width=3,
                              background_color='transparent'
                          ))
    arrow.add_to(map)

##### Generation of colors in the blue specter

In [ ]:
colormap = linear.YlGnBu_09.scale(0,118)
colors_dict = {i: colormap(i) for i in range(0,118)}

##### Plotting of the arrows varying by time and quantity of people 

In [ ]:
# First, lets group the dataframe by postal code
df_grouped = pre_df.groupby('postal')

# We only want those zip codes inside of the city of Barcelona
# List of all the postal codes in the dataset
valid_postal_codes = [8907,8040,8908,8039,8038,8004,8014,8905,8028,8950,8034,8017,8004,8017,8015,8021,8006,8029,8036,8008,8007,8011,8010,8009,8037,8013,8025,8026,8003,8005,8018,8019,8020,8027,8030,8016,8042,8031,8032,8035,8023,8022,8196,8198,8930,8918,8913,8923,8922,8914,8912]

# Remove all groups whose postal code is not in the list
df_grouped = [(postal, group) for postal, group in df_grouped if postal in valid_postal_codes]

# Now, for each starting postal code, group by destination center
for postal, df_postal in df_grouped:
    df_postal_grouped = df_postal.groupby('center')

    # Count ammount of rows in each group, get the maxim value
    maximum = df_postal_grouped.size().max()

    # Make a list with the x top centers
    top_centers = df_postal_grouped.size().sort_values(ascending=False).index[:1]
    
    for center, df_center in df_postal_grouped:
        if center not in top_centers:
            continue
        # Finally, let's draw the arrows
        start_point = [df_center['Latitude'].iloc[0], df_center['Longitude'].iloc[0]]

        velos_actius = (4.5 + 11) / 2
        velos_publics = (15 + 30) / 2
        velos_priv = 20

        temps = df_center["Distance"] / ((velos_actius * df_center["active_ratio"]) + (velos_publics * df_center["public_ratio"]) + (velos_priv * df_center["private_ratio"] ))
        mitjana_temps = temps.mean()
        max_temps = max(temps)
        min_temps=  min(temps)
        # Get the name of the center and get the coordinates from the dictionary
        end_point = dictionary_coord[df_center['center'].iloc[0]]
        #print((df_center.count()[0]))
        # Calculate the size of the arrow with a function of the ammount of people in comparison to the maximum
        arrow_scale = (df_center.shape[0] * df_center.count()[0] )/maximum
        hex = rgb2hex(colors_dict[(round(mitjana_temps* 100))]) 
        arrow_color = int(hex[1:],16)
        # Draw the arrow
        draw_arrow(map_multiPoints, start_point, end_point, arrow_scale=arrow_scale, arrow_color=arrow_color)



##### Generate the map

In [ ]:
# Save the map as an HTML file
map_multiPoints.save('map_multiPoints.html')